## 1. Configuração e Imports

In [34]:
# Imports essenciais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb
import pickle
import os
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.metrics.pairwise import euclidean_distances
import xgboost as xgb

# Configurações
plt.style.use('default')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 8)
np.random.seed(42)

## 2. Classe Principal

In [35]:
class Pipeline:
    
    def __init__(self, data_path='../data/', models_path='../models/'):
        self.data_path = data_path
        self.models_path = models_path
        self.timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Criando pasta de modelos se não existir
        os.makedirs(self.models_path, exist_ok=True)
        
        # Containers para dados e modelos
        self.raw_data = {}
        self.processed_data = {}
        self.models = {}
        self.encoders = {}
        self.feature_columns = []
    
    def load_raw_data(self):
        print("Carregando dados CSV")
        
        try:
            self.raw_data['consumo'] = pd.read_csv(f'{self.data_path}consumo.csv')
            self.raw_data['clima'] = pd.read_csv(f'{self.data_path}clima.csv')
            self.raw_data['clientes'] = pd.read_csv(f'{self.data_path}clientes.csv')
            
            # Convertendo datas
            self.raw_data['consumo']['date'] = pd.to_datetime(self.raw_data['consumo']['date'])
            self.raw_data['clima']['date'] = pd.to_datetime(self.raw_data['clima']['date'])
            
            print(f"Dados carregados - Consumo: {len(self.raw_data['consumo']):,}, Clima: {len(self.raw_data['clima']):,}, Clientes: {len(self.raw_data['clientes']):,}")
            
        except Exception as e:
            print(f"Erro: {str(e)}")
            raise
    
    def process_data(self):
        print("Tratando dados...")
        
        # Copiando dados para processamento
        clima_tratado = self.raw_data['clima'].copy()
        clientes_tratado = self.raw_data['clientes'].copy()
        consumo_tratado = self.raw_data['consumo'].copy()
        
        # Tratamento de valores faltantes de temperatura
        print("Imputando valores faltantes de temperatura...")
        clima_tratado = self._imputar_temperatura_regional(clima_tratado)
        
        # Tratamento de clientes com região desconhecida
        print("Imputando regiões desconhecidas...")
        clientes_tratado = self._imputar_regioes_desconhecidas(clientes_tratado, consumo_tratado)
        
        # Salvando dados tratados
        self.processed_data['clima'] = clima_tratado
        self.processed_data['clientes'] = clientes_tratado
        self.processed_data['consumo'] = consumo_tratado
        
        print("Dados tratados com sucesso!")
    
    def _imputar_temperatura_regional(self, df_clima):
        df_temp = df_clima.copy()
        valores_faltantes = df_temp['temperature'].isnull().sum()
        
        if valores_faltantes == 0:
            return df_temp
        
        for idx in df_temp[df_temp['temperature'].isnull()].index:
            regiao = df_temp.loc[idx, 'region']
            data = df_temp.loc[idx, 'date']
            
            # Buscar valores de temperatura da mesma região em um período de 7 dias
            inicio = data - pd.Timedelta(days=7)
            fim = data + pd.Timedelta(days=7)
            
            valores_proximos = df_temp[
                (df_temp['region'] == regiao) & 
                (df_temp['date'] >= inicio) & 
                (df_temp['date'] <= fim) & 
                (df_temp['temperature'].notna())
            ]['temperature']
            
            if len(valores_proximos) > 0:
                df_temp.loc[idx, 'temperature'] = valores_proximos.mean()
            else:
                # Se não houver valores próximos, usar a média geral da região
                media_regiao = df_temp[df_temp['region'] == regiao]['temperature'].mean()
                df_temp.loc[idx, 'temperature'] = media_regiao
        
        return df_temp
    
    def _imputar_regioes_desconhecidas(self, df_clientes, df_consumo):
        # Calculando perfil de consumo por cliente
        perfil_consumo = df_consumo.groupby('client_id')['consumption_kwh'].agg([
            'mean', 'std', 'min', 'max', 'median'
        ]).reset_index()
        
        # Adicionando região aos perfis
        perfil_consumo = perfil_consumo.merge(df_clientes, on='client_id', how='left')
        
        # Separando clientes conhecidos e desconhecidos
        perfil_conhecidos = perfil_consumo[perfil_consumo['region'] != 'Desconhecida'].copy()
        perfil_desconhecidos = perfil_consumo[perfil_consumo['region'] == 'Desconhecida'].copy()
        
        if len(perfil_desconhecidos) == 0:
            return df_clientes
        
        # Calculando perfil médio por região conhecida
        perfil_por_regiao = perfil_conhecidos.groupby('region')[['mean', 'std', 'min', 'max', 'median']].mean()
        
        # Imputando região para clientes desconhecidos
        df_clientes_tratado = df_clientes.copy()
        
        for idx, cliente in perfil_desconhecidos.iterrows():
            cliente_id = cliente['client_id']
            regiao_similar = self._encontrar_regiao_similar(cliente, perfil_por_regiao)
            
            # Atualizando a região
            df_clientes_tratado.loc[df_clientes_tratado['client_id'] == cliente_id, 'region'] = regiao_similar
        
        return df_clientes_tratado
    
    def _encontrar_regiao_similar(self, cliente_perfil, perfis_regionais):
        features = ['mean', 'std', 'min', 'max', 'median']
        
        # Preparando os dados para normalização
        dados_completos = pd.concat([
            perfis_regionais[features],
            cliente_perfil[features].to_frame().T
        ], ignore_index=True)
        
        # Normalizando os dados
        scaler = MinMaxScaler()
        dados_normalizados = scaler.fit_transform(dados_completos)
        
        # Separando dados normalizados
        perfis_norm = dados_normalizados[:-1]
        cliente_norm = dados_normalizados[-1:]
        
        # Calculando distâncias
        distancias_array = euclidean_distances(cliente_norm, perfis_norm)[0]
        
        # Criando dicionário de distâncias por região
        distancias = dict(zip(perfis_regionais.index, distancias_array))
        
        # Retornando região com menor distância
        return min(distancias, key=distancias.get)

pipeline = Pipeline()

## 3. Feature Engineering

In [36]:
def create_features_with_duckdb(pipeline):
    print("Criando features com DuckDB.")
    
    # Conectando ao DuckDB
    conn = duckdb.connect(':memory:')
    
    try:
        # Carregando dados processados
        consumo_df = pipeline.processed_data['consumo']
        clima_df = pipeline.processed_data['clima']
        clientes_df = pipeline.processed_data['clientes']
        
        # Criando tabelas no DuckDB
        conn.execute("DROP TABLE IF EXISTS consumo")
        conn.execute("DROP TABLE IF EXISTS clima")  
        conn.execute("DROP TABLE IF EXISTS clientes")
        
        # Tabela de Clientes
        conn.execute("""
            CREATE TABLE clientes (
                client_id VARCHAR PRIMARY KEY,
                region VARCHAR NOT NULL
            );
        """)
        conn.execute("INSERT INTO clientes SELECT * FROM clientes_df")
        
        # Tabela de Clima
        conn.execute("""
            CREATE TABLE clima (
                region VARCHAR NOT NULL,
                date DATE NOT NULL,
                temperature DOUBLE NOT NULL,
                humidity DOUBLE NOT NULL,
                PRIMARY KEY (region, date)
            );
        """)
        conn.execute("INSERT INTO clima SELECT * FROM clima_df")
        
        # Tabela de Consumo
        conn.execute("""
            CREATE TABLE consumo (
                client_id VARCHAR NOT NULL,
                date DATE NOT NULL,
                consumption_kwh DOUBLE NOT NULL,
                PRIMARY KEY (client_id, date),
                FOREIGN KEY (client_id) REFERENCES clientes(client_id)
            );
        """)
        conn.execute("INSERT INTO consumo SELECT * FROM consumo_df")
        
        # Criando dataset com features
        print("Criando features avançadas.")
        
        dataset_final = conn.execute("""
            SELECT 
                co.client_id,
                co.date,
                co.consumption_kwh as target,
                cl.temperature,
                cl.humidity,
                -- Calculando sensação térmica (Heat Index)
                CASE 
                    WHEN ((cl.temperature * 9/5) + 32) < 80 THEN cl.temperature
                    ELSE ROUND(
                        ((-42.379 + 
                          2.04901523 * ((cl.temperature * 9/5) + 32) + 
                          10.14333127 * cl.humidity - 
                          0.22475541 * ((cl.temperature * 9/5) + 32) * cl.humidity - 
                          6.83783e-3 * POWER(((cl.temperature * 9/5) + 32), 2) - 
                          5.481717e-2 * POWER(cl.humidity, 2) + 
                          1.22874e-3 * POWER(((cl.temperature * 9/5) + 32), 2) * cl.humidity + 
                          8.5282e-4 * ((cl.temperature * 9/5) + 32) * POWER(cl.humidity, 2) - 
                          1.99e-6 * POWER(((cl.temperature * 9/5) + 32), 2) * POWER(cl.humidity, 2)
                        ) - 32) * 5/9, 2)
                END as sensacao_termica,
                c.region,
                EXTRACT(YEAR FROM co.date) as year,
                EXTRACT(MONTH FROM co.date) as month,
                EXTRACT(DAY FROM co.date) as day,
                EXTRACT(DOW FROM co.date) as day_of_week,
                EXTRACT(DOY FROM co.date) as day_of_year,
                LAG(co.consumption_kwh, 1) OVER (PARTITION BY co.client_id ORDER BY co.date) as consumption_lag1,
                LAG(co.consumption_kwh, 7) OVER (PARTITION BY co.client_id ORDER BY co.date) as consumption_lag7,
                AVG(co.consumption_kwh) OVER (PARTITION BY co.client_id ORDER BY co.date ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as consumption_ma7
            FROM consumo co
            JOIN clientes c ON co.client_id = c.client_id
            JOIN clima cl ON c.region = cl.region AND co.date = cl.date
            ORDER BY co.client_id, co.date
        """).fetchdf()
        
        # Removendo registros sem lag features (início das séries)
        dataset_final = dataset_final[dataset_final['consumption_lag1'].notna()].reset_index(drop=True)
        
        pipeline.processed_data['dataset_final'] = dataset_final
        
        print(f"Features criadas - Dataset final: {len(dataset_final):,} registros")
        
        # Salvando dataset para uso posterior
        dataset_final.to_csv(f'{pipeline.data_path}dataset_modelagem.csv', index=False)
        
    finally:
        conn.close()

# Adicionando método à classe
pipeline.create_features = lambda: create_features_with_duckdb(pipeline)

## 4. Treinamento de Modelos

In [37]:
def train_models(pipeline):
    print("Treinando modelos...")
    
    # Preparando dados
    df = pipeline.processed_data['dataset_final'].copy()
    df_sorted = df.sort_values(['client_id', 'date']).reset_index(drop=True)
    
    # Definindo features - usando apenas as features de lag/média móvel
    feature_columns = [
        'consumption_lag1', 'consumption_lag7', 'consumption_ma7'
    ]
    pipeline.feature_columns = feature_columns
    
    X = df_sorted[feature_columns]
    y = df_sorted['target']
    
    # Divisão temporal dos dados
    total_records = len(df_sorted)
    train_size = int(0.7 * total_records)
    val_size = int(0.15 * total_records)
    
    X_train = X.iloc[:train_size]
    y_train = y.iloc[:train_size]
    X_val = X.iloc[train_size:train_size + val_size]
    y_val = y.iloc[train_size:train_size + val_size]
    X_test = X.iloc[train_size + val_size:]
    y_test = y.iloc[train_size + val_size:]
    
    print(f"Divisão dos dados - Treino: {len(X_train):,}, Val: {len(X_val):,}, Teste: {len(X_test):,}")
    
    # Função para calcular métricas
    def calcular_metricas(y_true, y_pred):
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        
        # MAPE
        epsilon = 1e-10
        mape = np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100
        
        return {'RMSE': rmse, 'MAE': mae, 'MAPE': mape, 'R2': r2}
    
    # TimeSeriesSplit para validação
    tscv = TimeSeriesSplit(n_splits=3)
    
    # Random Forest
    print("Treinando Random Forest...")
    
    rf_param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [10, 20],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2]
    }
    
    rf_grid = GridSearchCV(
        RandomForestRegressor(random_state=42, n_jobs=-1),
        rf_param_grid,
        cv=tscv,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )
    
    rf_grid.fit(X_train, y_train)
    rf_model = rf_grid.best_estimator_
    
    # Predições Random Forest
    rf_train_pred = rf_model.predict(X_train)
    rf_val_pred = rf_model.predict(X_val)
    rf_test_pred = rf_model.predict(X_test)
    
    rf_metrics = {
        'train': calcular_metricas(y_train, rf_train_pred),
        'val': calcular_metricas(y_val, rf_val_pred),
        'test': calcular_metricas(y_test, rf_test_pred)
    }
    
    # XGBoost
    print("Treinando XGBoost...")
    
    xgb_param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [3, 6],
        'learning_rate': [0.1, 0.2],
        'subsample': [0.8, 1.0]
    }
    
    xgb_grid = GridSearchCV(
        xgb.XGBRegressor(random_state=42, n_jobs=-1, verbosity=0),
        xgb_param_grid,
        cv=tscv,
        scoring='neg_mean_squared_error',
        n_jobs=-1
    )
    
    xgb_grid.fit(X_train, y_train)
    xgb_model = xgb_grid.best_estimator_
    
    # Predições XGBoost
    xgb_train_pred = xgb_model.predict(X_train)
    xgb_val_pred = xgb_model.predict(X_val)
    xgb_test_pred = xgb_model.predict(X_test)
    
    xgb_metrics = {
        'train': calcular_metricas(y_train, xgb_train_pred),
        'val': calcular_metricas(y_val, xgb_val_pred),
        'test': calcular_metricas(y_test, xgb_test_pred)
    }
    
    # Comparando modelos e selecionando o melhor
    rf_test_rmse = rf_metrics['test']['RMSE']
    xgb_test_rmse = xgb_metrics['test']['RMSE']
    
    if rf_test_rmse < xgb_test_rmse:
        best_model = rf_model
        best_model_name = 'Random Forest'
        best_metrics = rf_metrics
        best_predictions = rf_test_pred
    else:
        best_model = xgb_model
        best_model_name = 'XGBoost'
        best_metrics = xgb_metrics
        best_predictions = xgb_test_pred
    
    # Salvando modelos
    pipeline.models = {
        'best_model': best_model,
        'best_model_name': best_model_name,
        'random_forest': rf_model,
        'xgboost': xgb_model,
        'metrics': {
            'random_forest': rf_metrics,
            'xgboost': xgb_metrics,
            'best': best_metrics
        },
        'test_data': {
            'X_test': X_test,
            'y_test': y_test,
            'predictions': best_predictions
        }
    }
    
    print(f"Melhor modelo: {best_model_name} (RMSE: {best_metrics['test']['RMSE']:.4f})")
    
    # Exibindo métricas
    print(f"Random Forest - Teste RMSE: {rf_metrics['test']['RMSE']:.4f}")
    print(f"XGBoost - Teste RMSE: {xgb_metrics['test']['RMSE']:.4f}")
    print(f"Melhor modelo: {best_model_name}")
    print(f"RMSE: {best_metrics['test']['RMSE']:.4f}")
    print(f"MAE: {best_metrics['test']['MAE']:.4f}")
    print(f"MAPE: {best_metrics['test']['MAPE']:.2f}%")
    print(f"R²: {best_metrics['test']['R2']:.4f}")

# Adicionando método à classe
pipeline.train_models = lambda: train_models(pipeline)

## 5. Versionamento e Registro

In [38]:
def save_models_and_metadata(pipeline):
    print("Salvando modelo...")
    
    timestamp = pipeline.timestamp
    
    # Salvando pipeline
    complete_pipeline = {
        'model': pipeline.models['best_model'],
        'feature_columns': pipeline.feature_columns,
        'model_name': pipeline.models['best_model_name'],
        'metrics': pipeline.models['metrics']['best'],
        'timestamp': timestamp
    }
    
    pipeline_path = f'{pipeline.models_path}pipeline_{timestamp}.pkl'
    with open(pipeline_path, 'wb') as f:
        pickle.dump(complete_pipeline, f)
    
    print(f"Modelo salvo: pipeline_{timestamp}.pkl")
    
    return complete_pipeline

# Adicionando método à classe
pipeline.save_models = lambda: save_models_and_metadata(pipeline)

## 6. Função de Inferência

In [39]:
def create_inference_system(pipeline):
    
    def predict_consumption(client_id, date, consumption_lag1, consumption_lag7, consumption_ma7):    
        
        # Convertendo data se necessário
        if isinstance(date, str):
            date = pd.to_datetime(date)
        
        # Montando array de features - apenas as 3 features de lag/média móvel
        features = np.array([[
            consumption_lag1, consumption_lag7, consumption_ma7
        ]])
        
        # Fazendo predição
        prediction = pipeline.models['best_model'].predict(features)[0]
        
        return {
            'client_id': client_id,
            'date': date.strftime('%Y-%m-%d'),
            'predicted_consumption': round(prediction, 2),
            'model_used': pipeline.models['best_model_name'],
            'input_features': {
                'consumption_lag1': consumption_lag1,
                'consumption_lag7': consumption_lag7,
                'consumption_ma7': consumption_ma7
            }
        }
    
    pipeline.predict = predict_consumption
    print("Sistema de inferência configurado!")

# Adicionando método à classe
pipeline.create_inference = lambda: create_inference_system(pipeline)

## 7. Execução do Pipeline Completo

In [40]:
# Executando o pipeline completo
print("Iniciando Pipeline")

try:
    # Etapa 1: Ingestão de dados
    pipeline.load_raw_data()
    
    # Etapa 2: Tratamento de dados
    pipeline.process_data()
    
    # Etapa 3: Feature engineering
    pipeline.create_features()
    
    # Etapa 4: Treinamento de modelos
    pipeline.train_models()
    
    # Etapa 5: Versionamento e salvamento
    saved_pipeline = pipeline.save_models()
    
    # Etapa 6: Sistema de inferência
    pipeline.create_inference()
        
    print(f"ID da execução: {pipeline.timestamp}")
    print(f"Melhor modelo: {pipeline.models['best_model_name']}")
    print(f"RMSE no teste: {pipeline.models['metrics']['best']['test']['RMSE']:.4f}")
    
except Exception as e:
    print(f"Erro: {str(e)}")
    raise

Iniciando Pipeline
Carregando dados CSV
Dados carregados - Consumo: 18,000, Clima: 900, Clientes: 100
Tratando dados...
Imputando valores faltantes de temperatura...
Imputando regiões desconhecidas...
Dados tratados com sucesso!
Criando features com DuckDB.
Criando features avançadas.
Features criadas - Dataset final: 17,900 registros
Treinando modelos...
Divisão dos dados - Treino: 12,530, Val: 2,685, Teste: 2,685
Treinando Random Forest...
Treinando XGBoost...
Melhor modelo: XGBoost (RMSE: 2.0213)
Random Forest - Teste RMSE: 2.0420
XGBoost - Teste RMSE: 2.0213
Melhor modelo: XGBoost
RMSE: 2.0213
MAE: 1.6066
MAPE: 11.43%
R²: 0.7096
Salvando modelo...
Modelo salvo: pipeline_20250813_204748.pkl
Sistema de inferência configurado!
ID da execução: 20250813_204748
Melhor modelo: XGBoost
RMSE no teste: 2.0213


## 8. Inferência direta

In [41]:
# Testando inferência
# Exemplo 1: Previsão com dados históricos
resultado1 = pipeline.predict(
    client_id='CLIENT_001',
    date='2024-01-15',
    consumption_lag1=23.5,
    consumption_lag7=21.2,
    consumption_ma7=22.8
)

print("Exemplo 1")
print(f"Cliente: {resultado1['client_id']}")
print(f"Data: {resultado1['date']}")
print(f"Consumo previsto: {resultado1['predicted_consumption']} kWh")
print(f"Features usadas: {resultado1['input_features']}")
print(f"Modelo usado: {resultado1['model_used']}")
print()

# Exemplo 2: Previsão com outro padrão de consumo
resultado2 = pipeline.predict(
    client_id='CLIENT_002',
    date='2024-06-20',
    consumption_lag1=15.3,
    consumption_lag7=16.7,
    consumption_ma7=14.5
)

print("Exemplo 2")
print(f"Cliente: {resultado2['client_id']}")
print(f"Data: {resultado2['date']}")
print(f"Consumo previsto: {resultado2['predicted_consumption']} kWh")
print(f"Features usadas: {resultado2['input_features']}")
print(f"Modelo usado: {resultado2['model_used']}")
print()

# Exemplo 3: Previsão com padrão de alto consumo
resultado3 = pipeline.predict(
    client_id='CLIENT_003',
    date='2024-08-10',
    consumption_lag1=28.0,
    consumption_lag7=26.5,
    consumption_ma7=27.2
)

print("Exemplo 3")
print(f"Cliente: {resultado3['client_id']}")
print(f"Data: {resultado3['date']}")
print(f"Consumo previsto: {resultado3['predicted_consumption']} kWh")
print(f"Features usadas: {resultado3['input_features']}")
print(f"Modelo usado: {resultado3['model_used']}")

Exemplo 1
Cliente: CLIENT_001
Data: 2024-01-15
Consumo previsto: 21.059999465942383 kWh
Features usadas: {'consumption_lag1': 23.5, 'consumption_lag7': 21.2, 'consumption_ma7': 22.8}
Modelo usado: XGBoost

Exemplo 2
Cliente: CLIENT_002
Data: 2024-06-20
Consumo previsto: 14.550000190734863 kWh
Features usadas: {'consumption_lag1': 15.3, 'consumption_lag7': 16.7, 'consumption_ma7': 14.5}
Modelo usado: XGBoost

Exemplo 3
Cliente: CLIENT_003
Data: 2024-08-10
Consumo previsto: 20.93000030517578 kWh
Features usadas: {'consumption_lag1': 28.0, 'consumption_lag7': 26.5, 'consumption_ma7': 27.2}
Modelo usado: XGBoost
